In [ ]:
# Call Places API

# Import packages 
library("httr") 
library("jsonlite")  
library('gtools') 

# Define API parameters 
key <- 'Insert API Key'
query <- 'EX4' 


# Define empty dataframe anda offset. Change second value to increase number of results. 
offset= seq(0, 900, by=100) 
dataframe = data.frame() 

# Make API requests with offsets and concatenate results 
for (value in offset) { 
  url <- paste0('https://api.os.uk/search/places/v1/find?query=',query,'&offset=',value) 
  data <- GET(url,add_headers(key=key)) 
  json <- fromJSON(rawToChar(data$content), flatten=TRUE)  
  dataframe <- smartbind(dataframe,json$results) 
} 


In [ ]:
# Call Maps API for basemapping
OSRoad_3857 <- paste0("https://api.os.uk/maps/raster/v1/zxy/Road_3857/{z}/{x}/{y}.png?key=", key)

In [ ]:
# Call features API

# Import required libraries
library(shiny)
library(shinydashboard)
library(leaflet)
library(sf)
library(jsonlite)
library(httr)
library(DT) # Added library for DataTables

# Define maximum features
maxfeatures <- 2000

# Create empty data frame
df_total <- data.frame()

# Input parameters for bbox and key
bbox <- "-3.180600,51.460534,-3.169943,51.467383"
key <- "Insert API Key"
collection_name <- "bld-fts-buildingpart"

# Call Maps API for basemapping
OSRoad_3857 <- paste0("https://api.os.uk/maps/raster/v1/zxy/Road_3857/{z}/{x}/{y}.png?key=", key)

# Return JSON data from API
url <- paste0("https://api.os.uk/features/ngd/ofa/v1/collections/", collection_name, "/items?bbox=", bbox)
sf <- read_sf(GET(url, crs = 27700, add_headers(key = key)), as_tibble = FALSE)
rawdata <- GET(url, add_headers(key = key))
json <- fromJSON(rawToChar(rawdata$content), flatten = TRUE)

# Add first 100 features from API response to data frame
df_total <- rbind(df_total, sf)

# Page through API binding to data frame until maximum features reached or all features collected
while (TRUE) {
  if ("Next page" == tail(json$links$title, n = 1) & maxfeatures > nrow(df_total)) {
    url_iter <- tail(json$links$href, n = 1)
    raw_data2 <- GET(url_iter, add_headers(key = key))
    json <- fromJSON(rawToChar(raw_data2$content), flatten = TRUE)
    sf2 <- read_sf(GET(url_iter, add_headers(key = key)), as_tibble = FALSE)
    df_total <- rbind(df_total, sf2)
  } else {
    break
  }
}

# Define app server functions for shiny
server <- function(input, output, session) {
  output$map <- renderLeaflet({
    leaflet() %>%
      addTiles(OSRoad_3857) %>% 
      setView(lng = -3.177899, lat = 51.461884, zoom = 12) %>% 
      addPolygons(data = df_total, 
                  layerId = ~id, # Assign a unique layer ID to each polygon
                  popup = ~paste("TOID", toid)) # Customize the popup content with feature attributes
  })
  
  output$table <- renderDataTable({
    df_total
  }, options = list(scrollX = TRUE)) # Enable horizontal scrolling in the table
  
}

# Define app front-end
ui <- dashboardPage(
  dashboardHeader(title = "My Dashboard"),
  dashboardSidebar(),
  dashboardBody(
    fluidRow(
      box(
        title = "Map",
        leafletOutput("map", height = 600)
      )
    ),
    fluidRow(
      box(
        title = "Data Table",
        dataTableOutput("table", width = "100%") # Set the table width to 100%
      )
    )
  )
)

# Run app
shinyApp(ui, server)